![表紙](https://www.oreilly.co.jp/books/images/picture978-4-87311-907-6.gif)

このノートブックはオライリー・ジャパンより発行の書籍[『セキュリティエンジニアのための機械学習』](https://www.oreilly.co.jp/books/9784873119076/)のサンプルコードです。コードの解説等は書籍をご参照ください。なお、このコードを動作させた結果について、著者およびオライリー・ジャパンは一切の責任を負いません。

In [ ]:
import tweepy
import json
import numpy as np
import pandas as pd

# APIを使用するための鍵
consumer_key="YOUR KEY"
consumer_secret="YOUR KEY"
access_token="YOUR KEY"
access_token_secret="YOUR KEY"

# TweepyによるTwitter APIを使用するための認証のセットアップ
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# パラメータwait_on_rate_limitを有効化して、 APIの問い合わせ回数の上限に達した場合は必要時間だけ待機
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# 検索ワードとして exploit http を使用し、リツイートは除外
search_term = "exploit http -filter:retweets"

# 一番古いツイートidを保存するための変数
oldest_tweet = None

# 回収したツイートの一覧を格納するためのリスト
TempDict = []

# 回収したツイート数のカウンター
counter = 0

# １０回ループし、合計1000件のツイートを対象にする
for x in range(10):

    # 最新のツイートから100件を抽出し、テキストをすべて収集する
    public_tweets = api.search(search_term, 
                               count=100, 
                               result_type="recent",
                               tweet_mode="extended", 
                               max_id=oldest_tweet)

    # 条件に一致するツイートの収集
    for tweet in public_tweets["statuses"]:
        #　引用リツイートも除外
        if not 'quoted_status' in tweet:
            TempDict.append(tweet)
                            
            # カウンターに１を追加
            counter += 1

        # 検索結果の一番古いツイートidを代入し、次の検索結果はこの一番古いid
        # より古いものだけを対象にする
        oldest_tweet = tweet["id"]

print("Tweet {}件を収集しました".format(counter))

Tweet 451件を収集しました


In [ ]:
data = pd.DataFrame(data=[tweet['full_text'] for tweet in TempDict], columns=['TweetText'])

In [ ]:
import re
URLPATTERN = r'(https?://\S+)'

data['URL'] = data["TweetText"].str.extract(URLPATTERN, expand=False).str.strip()

In [ ]:
# ツイートしたユーザーのフォロワー数の追加
data['followers_count'] = \
pd.DataFrame(data=[tweet['user']['followers_count'] \
                   for tweet in TempDict])

In [ ]:
# ツイートしたユーザーの「いいね！」された数の追加
data['favourites_count'] = \
pd.DataFrame(data=[tweet['user']['favourites_count'] \
                   for tweet in TempDict])

In [ ]:
# ツイートしたユーザーが認証済みアカウントであるか、そうでないかの情報を追加
data['verified'] = \
pd.DataFrame(data=[tweet['user']['verified'] \
                   for tweet in TempDict])

In [ ]:
data.head(20)

,TweetText,URL,followers_count,favourites_count,verified
0,OL : L'exploit face à Man City élu match de l'...,https://t.co/Ih2fyz2b9j,173,31,False
1,Republican efforts to undermine Biden victory ...,https://t.co/h8r9kOXBuY,19,1031,False
2,The scattershot efforts to overturn President-...,https://t.co/BubYeURzb2,33,3,False
3,"@MSNBC Only goes to show, how #DonaldTrump wil...",https://t.co/PwzukzW05L,2296,53805,False
4,exploit frictionless #supply-chains with #doma...,https://t.co/qTvwSQLAvj,1245,220,False
5,#DeFi insurer @CoverProtocol was exploited for...,https://t.co/hlS98EtpPw,4177,191,False
6,Thanks for TAB message format. I did some expl...,https://t.co/SZUogOi0Lj,27,1,False
7,"I ruin credit scores, marriages &amp; manipula...",https://t.co/NYQPND6xUo,5503,14959,False
8,Thinking of publishing the Next Exploit source...,https://t.co/mo06EGORFw,1,5,False
9,Cyberpunk 2077 speedrunners have found a bunny...,https://t.co/LkLIuuc5Lq,270382,4467,False
